# Bluesky Subscriptions

The RunEngine generates Documents, Python dictionaries containing metadata and data. The RunEngine maintains a list of "callbacks" that consume these Documents.

In [0]:
%matplotlib notebook
from bluesky.examples import motor, det
from bluesky.run_engine import RunEngine
from bluesky.plans import AbsScanPlan
from bluesky.callbacks import LiveTable, LivePlot

In [0]:
RE = RunEngine({'owner': 'demo', 'group': 'demo', 'beamline_id': 'demo'})

In [0]:
plan = AbsScanPlan([det], motor, 1, 5, 5)

This moves the motor and reads from the motor and the detector, but the data goes nowhere.

In [0]:
RE(plan)

## Three ways to add a callback

### 1. Use a callback just for one run.

In [0]:
RE(plan, LiveTable(['det']))

### 2. Associate the callback with the `plan` object itself.

In [0]:
plan.subs = LiveTable(['det'])

In [0]:
RE(plan)

A list of callbacks is also accepted.

In [0]:
plan.subs = [LiveTable(['det']), LivePlot('det')]
# Calling LivePlot creates the figure. When a run a scan with ascan, it will be filled with data.

In [0]:
RE(plan)

In [0]:
# Run again -- with slightly different settings -- and see the plot above updated.
plan.num = 3
RE(plan)

### 3. Run a callback on all runs by associating it with `RE` itself.

This is less common. Notice that, in this case, we require you to specify which kinds of Documents should be sent to the callback: start, stop, descriptor, event, or all.

```
RE.subscribe('all', some_callback)
```

## Writing custom callbacks

In [0]:
def my_callback(name, doc):
    "a very simple custom callback"
    if name == 'start':
        print('My scan id is', doc['scan_id'])
    elif name == 'event':
        print('This is event number', doc['seq_num'])
    elif name == 'stop':
        print('We are done')

In [0]:
RE(plan, my_callback)

A utility class, `CallbackBase`, is provided to handle the boilerplate dispatching of different document types.

In [0]:
from bluesky.callbacks import CallbackBase

class MyLiveCallback(CallbackBase):
    """
    The CallbackBase class dispatches each kind of document
    to an associated method.
    """
    
    def start(self, doc):
        print(doc)
    
    def event(self, doc):
        print(doc)
    
    def descriptor(self, doc):
        print(doc)
    
    def stop(self, doc):
        print(doc)

In [0]:
RE(plan, MyLiveCallback())